## GPU Configuration

In [6]:
!nvidia-smi

In [ ]:
%load_ext cudf.pandas

## Loading Datasets

In [ ]:
import pandas as pd

df_price = pd.read_json('../data/aapl_price.json')

In [ ]:
df_price

# Resources

* [https://www.kaggle.com/code/yasirabdaali/make-pandas-super-fast-with-gpu-acceleration](https://www.kaggle.com/code/yasirabdaali/make-pandas-super-fast-with-gpu-acceleration)